In [320]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import numpy as np

data = pd.read_csv("donnees_finales_2.xlsx - Sheet1.csv")
data = data.drop(columns=['idstd'])

# Normalisation de la colonne 'anciennete'
scaler = StandardScaler()
if 'anciennete' in data.columns:
    data['anciennete'] = scaler.fit_transform(data[['anciennete']])

data['colab_info'] = data['collaboration'] * data['informatique']
data['colab_RD'] = data['collaboration'] * data['recherche_developpement']
data['info_RD'] = data['informatique'] * data['recherche_developpement']
data['colab_info_RD'] = data['collaboration'] * data['informatique'] * data['recherche_developpement']


# Filtrer les données pour le pays Inde
data_inde = data.loc[data['pays'] == 'India']
data =  data.drop(columns=['pays'])

idées:
- variable = regression poisson
- variable comptage * ancienneté

In [322]:
X = data.drop(columns=['innovation_produits_services','innovation_procedes','innovation_organisation',
              'innovation_marketing','innovation','innovation_comptage'])
X_inde = data_inde.drop(columns=['innovation_produits_services','innovation_procedes','innovation_organisation',
              'innovation_marketing','innovation','innovation_comptage', 'pays_NE','pays_BA','pays_IN','pays_PA','pays'])
y_bin = data['innovation']
y_cpt = data['innovation_comptage']

y_inde_bin = data_inde['innovation']
y_inde_cpt = data_inde['innovation_comptage']

Modele poisson

In [323]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculer le VIF pour chaque variable explicative
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

                   Variable        VIF
0              informatique   4.307612
1   recherche_developpement   1.766147
2             collaboration   9.449537
3          secteur_services   1.283822
4          secteur_produits   7.213595
5                anciennete   1.032996
6              taille_grand  25.475734
7              taille_moyen  32.242931
8              taille_petit  29.745044
9                   pays_NE  12.573726
10                  pays_BA  33.591704
11                  pays_IN  87.456474
12                  pays_PA  15.502830
13               colab_info   9.838058
14                 colab_RD   9.526470
15                  info_RD   4.662349
16            colab_info_RD  10.028439


beaucoup de variable avec un VIF élevé, il vaut mieux utiliser une penalité Lasso pour résoudre simultanément les problèmes de multicolinéarité et de sélection de variables.

Reg poisson avec Lasso 

In [324]:
from pyglmnet import GLM
import statsmodels.api as sm
import pandas as pd
import numpy as np

# Étape 1 : Gestion de la colinéarité avec Lasso (pyglmnet)
# Préparation des données
X2 = data.drop(columns=['innovation_produits_services', 'innovation_procedes', 
                        'innovation_organisation', 'innovation_marketing', 
                        'innovation', 'innovation_comptage']).values
y_cpt2 = data['innovation_comptage'].values

# Ajouter une constante pour inclure un intercept
X2_with_const = np.hstack((np.ones((X2.shape[0], 1)), X2))

# Modèle de Poisson avec pénalisation L1 (Lasso)
glm = GLM(distr='poisson', alpha=1.0, reg_lambda=0.002)
glm.fit(X2_with_const, y_cpt2)

# Affichage des coefficients
columns = ['Intercept'] + list(X.columns)
coefficients_lasso = pd.DataFrame({
    'Variable': columns,
    'Coefficient': glm.beta_
})
print("Coefficients avec pénalisation Lasso :")
print(coefficients_lasso)

# Étape 2 : Modèle classique sans pénalisation pour significativité (statsmodels)
# Ré-ajuster un modèle Poisson sans pénalisation sur les variables sélectionnées
selected_features = coefficients_lasso[coefficients_lasso['Coefficient'] != 0]['Variable']
X_selected = data[selected_features[1:]]  # Exclure l'intercept
X_selected_const = sm.add_constant(X_selected)

# Ajuster un modèle Poisson sans pénalisation
poisson_model = sm.GLM(y_cpt2, X_selected_const, family=sm.families.Poisson()).fit()

# Résumé des résultats
print("\nRésumé du modèle classique (sans pénalisation) :")
print(poisson_model.summary())


Coefficients avec pénalisation Lasso :
                   Variable  Coefficient
0                 Intercept    -0.000000
1              informatique     0.019602
2   recherche_developpement     0.287383
3             collaboration     0.175361
4          secteur_services    -0.047636
5          secteur_produits    -0.000000
6                anciennete    -0.001421
7              taille_grand     0.049869
8              taille_moyen    -0.000000
9              taille_petit    -0.182307
10                  pays_NE     0.000000
11                  pays_BA     0.330601
12                  pays_IN     0.301095
13                  pays_PA    -0.324973
14               colab_info     0.000000
15                 colab_RD     0.000000
16                  info_RD    -0.000000
17            colab_info_RD     0.000000

Résumé du modèle classique (sans pénalisation) :
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observati

c:\Users\zoero\OneDrive\Bureau\M2\S1\innovation\projet\.venv\Lib\site-packages\pyglmnet\pyglmnet.py:863: UserWarning: Reached max number of iterations without convergence.
  warnings.warn(


In [326]:

# Préparation des données : convertir DataFrame et Series en ndarray
X_inde2 = data_inde.drop(columns=['innovation_produits_services','innovation_procedes','innovation_organisation',
              'innovation_marketing','innovation','innovation_comptage', 'pays_NE','pays_BA','pays_IN','pays_PA','pays']).values
y_inde_cpt2 = data_inde['innovation_comptage'].values  # Convertir en ndarray


# Ajouter une constante pour inclure un intercept
X_inde2_with_const = np.hstack((np.ones((X_inde2.shape[0], 1)), X_inde2))

# Modèle de Poisson avec pénalisation L1 (Lasso)
glm = GLM(distr='poisson', alpha=1.0, reg_lambda=0.002)
glm.fit(X_inde2_with_const, y_inde_cpt2)

# Affichage des coefficients
columns = ['Intercept'] + list(X_inde.columns)
coefficients_lasso = pd.DataFrame({
    'Variable': columns,
    'Coefficient': glm.beta_
})
print("Coefficients avec pénalisation Lasso :")
print(coefficients_lasso)

# Étape 2 : Modèle classique sans pénalisation pour significativité (statsmodels)
# Ré-ajuster un modèle Poisson sans pénalisation sur les variables sélectionnées
selected_features = coefficients_lasso[coefficients_lasso['Coefficient'] != 0]['Variable']
X_selected = data_inde[selected_features[1:]]  # Exclure l'intercept
X_selected_const = sm.add_constant(X_selected)

# Ajuster un modèle Poisson sans pénalisation
poisson_model = sm.GLM(y_inde_cpt2, X_selected_const, family=sm.families.Poisson()).fit()

# Résumé des résultats
print("\nRésumé du modèle classique (sans pénalisation) :")
print(poisson_model.summary())


Coefficients avec pénalisation Lasso :
                   Variable  Coefficient
0                 Intercept    -0.000000
1              informatique    -0.000000
2   recherche_developpement     0.156081
3             collaboration     0.155468
4          secteur_services    -0.061215
5          secteur_produits    -0.000000
6                anciennete    -0.020299
7              taille_grand     0.070150
8              taille_moyen    -0.000000
9              taille_petit    -0.204231
10               colab_info     0.000000
11                 colab_RD    -0.000000
12                  info_RD     0.002559
13            colab_info_RD     0.000000

Résumé du modèle classique (sans pénalisation) :
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 3492
Model:                            GLM   Df Residuals:                     3485
Model Family:                 Poisson   Df Model:          

Taux de diversité

In [329]:
y_bis = y_cpt * data['anciennete']
y_bis_inde = y_inde_cpt * data_inde['anciennete']

In [330]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm

# Étape 1 : Ajustement avec Lasso pour gérer la multicolinéarité et sélectionner les variables
lasso_model = make_pipeline(StandardScaler(), Lasso(alpha=0.002))  # Ajustez alpha selon vos besoins
lasso_model.fit(X, y_bis)

# Récupérer les coefficients après Lasso
coefficients = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': lasso_model.named_steps['lasso'].coef_
})
print("Coefficients avec pénalisation Lasso :")
print(coefficients)

# Étape 2 : Sélection des variables non nulles
selected_features = coefficients[coefficients['Coefficient'] != 0]['Variable']

# Étape 3 : Analyse classique avec statsmodels pour tester la significativité des variables sélectionnées
X_selected = X[selected_features]  # Conserver uniquement les variables sélectionnées
X_selected_const = sm.add_constant(X_selected)  # Ajouter une constante pour l'intercept

# Ajuster un modèle de régression linéaire classique
model = sm.OLS(y_bis, X_selected_const).fit()

# Résumé des résultats
print("\nRésumé du modèle classique (sans pénalisation) :")
print(model.summary())


Coefficients avec pénalisation Lasso :
                   Variable  Coefficient
0              informatique    -0.057853
1   recherche_developpement    -0.051164
2             collaboration    -0.000000
3          secteur_services     0.006130
4          secteur_produits     0.066070
5                anciennete     2.447283
6              taille_grand    -0.000000
7              taille_moyen    -0.006495
8              taille_petit     0.004161
9                   pays_NE     0.080681
10                  pays_BA    -0.044681
11                  pays_IN    -0.000000
12                  pays_PA     0.141908
13               colab_info    -0.024517
14                 colab_RD    -0.000000
15                  info_RD     0.081872
16            colab_info_RD     0.014223

Résumé du modèle classique (sans pénalisation) :
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.837
Model:         

In [331]:

lasso_model = make_pipeline(StandardScaler(), Lasso(alpha=0.002))  # Ajustez alpha selon vos besoins
lasso_model.fit(X_inde, y_bis_inde)

# Récupérer les coefficients après Lasso
coefficients = pd.DataFrame({
    'Variable': X_inde.columns,
    'Coefficient': lasso_model.named_steps['lasso'].coef_
})
print("Coefficients avec pénalisation Lasso :")
print(coefficients)

# Étape 2 : Sélection des variables non nulles
selected_features = coefficients[coefficients['Coefficient'] != 0]['Variable']

# Étape 3 : Analyse classique avec statsmodels pour tester la significativité des variables sélectionnées
X_selected = X_inde[selected_features]  # Conserver uniquement les variables sélectionnées
X_selected_const = sm.add_constant(X_selected)  # Ajouter une constante pour l'intercept

# Ajuster un modèle de régression linéaire classique
model = sm.OLS(y_bis_inde, X_selected_const).fit()

# Résumé des résultats
print("\nRésumé du modèle classique (sans pénalisation) :")
print(model.summary())

Coefficients avec pénalisation Lasso :
                   Variable  Coefficient
0              informatique     0.032484
1   recherche_developpement     0.010543
2             collaboration     0.014731
3          secteur_services     0.020232
4          secteur_produits     0.000000
5                anciennete     2.690180
6              taille_grand     0.018035
7              taille_moyen    -0.000000
8              taille_petit    -0.000000
9                colab_info    -0.054066
10                 colab_RD    -0.021455
11                  info_RD    -0.000000
12            colab_info_RD     0.051751

Résumé du modèle classique (sans pénalisation) :
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     3003.
Date:                